In [ ]:
import time, threading
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, TimeoutException
from shutil import which
from openpyxl import load_workbook

wb = load_workbook('BD_CADASTRO_NUMERADO_AGO_TESTE.xlsx')
ws, ws2 = wb['Fundos'], wb['Fundos_Cota']
lista_cnpj = []

for row in ws.iter_rows(values_only=True):
    cnpj = row[1]
    lista_cnpj.append(cnpj)

for row in ws2.iter_rows(values_only=True):
    cnpj = row[1] 
    lista_cnpj.append(cnpj)

print("Coleta de CNPJs finalizada")

dados_diarios_list = []

max_threads = 8
semaphore = threading.Semaphore(max_threads)


def count_itens_dropdown(cnpj):
    driver = init_driver("Dropdown")
    print("Coleta tamanho do dropdown iniciada")

    #pesquisa e troca para o frame correto sem precisar declarar uma variável nova no processo
    WebDriverWait(driver, 10).until(
    EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//frame[@name='Main']"))
    )

    search_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//input[@id='txtCNPJNome']"))
    )

    search_input.clear()
    #search_input.send_keys(lista_cnpj[0])
    search_input.send_keys(cnpj)
    search_input.send_keys(Keys.ENTER)

    try:
        find_name_to_store = WebDriverWait(driver, 1).until(
            EC.presence_of_element_located((By.XPATH, "//a[@id='ddlFundos__ctl1_Linkbutton4']"))
        )
    except TimeoutException:
        find_name_to_store = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//a[@id='ddlFundos__ctl0_lnkbtn1']"))
        )
    
    find_name_to_store.click()

    driver.switch_to.default_content()

    #pesquisa e troca para o frame correto sem precisar declarar uma variável nova no processo
    WebDriverWait(driver, 10).until(
        EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//frame[@name='Main']"))
    )

    dados_diarios = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//a[@id='Hyperlink2']"))
    )
    dados_diarios.click()

    driver.switch_to.default_content()

    #pop-up com mensagem irrelevante aparece e precisa ser fechado.
    try:
        alert = WebDriverWait(driver, 3).until(EC.alert_is_present())
        alert.accept()
    except:
        pass

    WebDriverWait(driver, 10).until(
        EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//frame[@name='Main']"))
    )

    dropdown_data_pesquisa = Select(WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//select[@name='ddComptc']"))
    ))

    print("Tamanho do dropdown coletado com sucesso")
    var_to_be_returned = len(dropdown_data_pesquisa.options)
    driver.quit()
    return var_to_be_returned


def init_driver(index):
    options = webdriver.ChromeOptions()

    options.add_argument("--headless=new")

    driver = webdriver.Chrome(executable_path=which('chromedriver'), options=options)
    driver.set_page_load_timeout(10)

    url='https://cvmweb.cvm.gov.br/swb/default.asp?sg_sistema=fundosreg'

    try:
        driver.get(url)
    except TimeoutException:
        print(f"Erro no carregamento da Thread {index+1}, reiniciando...")
        driver.quit()
        return init_driver(index)

    return driver

def get_info(cnpj, index):
    with semaphore:
        try:
            print(f"Thread {index+1} iniciada para o CNPJ {cnpj}")
            driver = init_driver(index)
            print(f"Driver da Thread {index+1} iniciado com sucesso")

            #pesquisa e troca para o frame correto sem precisar declarar uma variável nova no processo
            WebDriverWait(driver, 10).until(
            EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//frame[@name='Main']"))
            )

            search_input = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//input[@id='txtCNPJNome']"))
            )

            search_input.clear()
            #search_input.send_keys(lista_cnpj[0])
            search_input.send_keys(cnpj)
            search_input.send_keys(Keys.ENTER)
            print(f"Thread {index+1} pesquisou o CNPJ {cnpj}")

            try:
                find_name_to_store = WebDriverWait(driver, 1).until(
                    EC.presence_of_element_located((By.XPATH, "//a[@id='ddlFundos__ctl1_Linkbutton4']"))
                )
            except TimeoutException:
                find_name_to_store = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//a[@id='ddlFundos__ctl0_lnkbtn1']"))
                )

            stored_name = find_name_to_store.text
            find_name_to_store.click()

            driver.switch_to.default_content()

            #pesquisa e troca para o frame correto sem precisar declarar uma variável nova no processo
            WebDriverWait(driver, 10).until(
                EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//frame[@name='Main']"))
            )

            dados_diarios = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//a[@id='Hyperlink2']"))
            )
            dados_diarios.click()
            print(f"Thread {index+1} clicou em dados diários")

            driver.switch_to.default_content()

            #pop-up com mensagem irrelevante aparece e precisa ser fechado.
            try:
                alert = WebDriverWait(driver, 3).until(EC.alert_is_present())
                alert.accept()
            except:
                pass

            WebDriverWait(driver, 10).until(
                EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//frame[@name='Main']"))
            )

            dropdown_data_pesquisa = Select(WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//select[@name='ddComptc']"))
            ))

            print(f"Thread {index+1} iniciando coleta de dados diários")
            #dados_diarios_list = []
            
            driver.switch_to.default_content()

            WebDriverWait(driver, 10).until(
            EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//frame[@name='Main']"))
            )

            dropdown_data_pesquisa = Select(WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//select[@name='ddComptc']"))
            ))
            dropdown_data_pesquisa.select_by_index(index)

            name_selected_dropdown = Select(WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//select[@name='ddComptc']"))
            ))
            name_selected_dropdown = name_selected_dropdown.first_selected_option.get_attribute("value")
            
            linhas_tabela = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.XPATH, "//table[@id='dgDocDiario']//tr[position()>1]"))
            )
            linhas_validas = [linha for linha in linhas_tabela if linha.find_element(By.XPATH, "td[2]").text.strip()] 
            
            if linhas_validas:
                ultima_linha = linhas_validas[-1]
                dados = ultima_linha.find_elements(By.XPATH, "td")
                dados_diarios = {
                    "Cnpj": cnpj,
                    "Nome": stored_name,
                    "Mês": name_selected_dropdown,
                    "Dia": dados[0].text.strip(),
                    "Quota": dados[1].text.strip(),
                    #"Captação no Dia": dados[2].text.strip(),
                    #"Resgate no Dia": dados[3].text.strip(),
                    "Patrimônio Líquido": dados[4].text.strip(),
                    #"Total da Carteira": dados[5].text.strip(),
                    "Número de Cotistas": dados[6].text.strip(),
                    #"Data da Próxima Informação do PL": dados[7].text.strip(),
                }
                dados_diarios_list.append(dados_diarios)
        #dropdown.select_by_visible_text('02/2025')
            print(f"Thread {index+1} finalizou coleta de dados diários")
            driver.quit()
        except TimeoutException:
            print(f"Erro na Thread {index+1}")
            driver.quit()
            print(f"Thread {index+1} reiniciando...")
            get_info(cnpj, index)

list_cnpj_teste = ["58.878.941/0001-02", "36.248.874/0001-00", "51.017.442/0001-81", "57.270.020/0001-08"]

threads = []
tamanho = count_itens_dropdown(lista_cnpj[1])
to_int = int(tamanho)

# Criando threads em blocos de 8
for index in range(to_int):
    thread = threading.Thread(target=get_info, args=(lista_cnpj[1], index,))
    threads.append(thread)
    thread.start()

# Aguardando todas as threads terminarem
for thread in threads:
    thread.join()

print("Todas as Threads foram finalizadas")

Coleta de CNPJs finalizada
Coleta tamanho do dropdown iniciada
Tamanho do dropdown coletado com sucesso
Thread 1 iniciada para o CNPJ 13.077.415/0001-05
Thread 2 iniciada para o CNPJ 13.077.415/0001-05
Thread 3 iniciada para o CNPJ 13.077.415/0001-05
Thread 4 iniciada para o CNPJ 13.077.415/0001-05
Thread 5 iniciada para o CNPJ 13.077.415/0001-05
Thread 6 iniciada para o CNPJ 13.077.415/0001-05
Thread 7 iniciada para o CNPJ 13.077.415/0001-05
Thread 8 iniciada para o CNPJ 13.077.415/0001-05
Driver da Thread 1 iniciado com sucesso
Thread 1 pesquisou o CNPJ 13.077.415/0001-05
Driver da Thread 7 iniciado com sucesso
Driver da Thread 6 iniciado com sucesso
Driver da Thread 2 iniciado com sucesso
Thread 1 clicou em dados diários
Thread 7 pesquisou o CNPJ 13.077.415/0001-05
Thread 6 pesquisou o CNPJ 13.077.415/0001-05
Thread 2 pesquisou o CNPJ 13.077.415/0001-05
Erro no carregamento da Thread 8, reiniciando...
Thread 7 clicou em dados diários
Thread 1 iniciando coleta de dados diários
Thread

In [39]:
import pandas as pd
testing = pd.DataFrame(dados_diarios_list)
testing['Mês'] = pd.to_datetime(testing['Mês'], format='%m/%Y')
testing_sorted = testing.sort_values(by='Mês', ignore_index=True)

testing_sorted

,Cnpj,Nome,Mês,Dia,Quota,Patrimônio Líquido,Número de Cotistas
0,13.077.415/0001-05,BB PREVIDENCIÁRIO FLUXO RENDA FIXA SIMPLES FUN...,2011-04-01,29,"1,000378","1.156.043,65",6
1,13.077.415/0001-05,BB PREVIDENCIÁRIO FLUXO RENDA FIXA SIMPLES FUN...,2011-05-01,31,"1,009178","43.118.555,31",41
2,13.077.415/0001-05,BB PREVIDENCIÁRIO FLUXO RENDA FIXA SIMPLES FUN...,2011-06-01,30,"1,018104","51.178.462,87",68
3,13.077.415/0001-05,BB PREVIDENCIÁRIO FLUXO RENDA FIXA SIMPLES FUN...,2011-07-01,29,"1,027238","86.750.252,95",93
4,13.077.415/0001-05,BB PREVIDENCIÁRIO FLUXO RENDA FIXA SIMPLES FUN...,2011-08-01,31,"1,037549","127.852.265,60",111
...,...,...,...,...,...,...,...
162,13.077.415/0001-05,BB PREVIDENCIÁRIO FLUXO RENDA FIXA SIMPLES FUN...,2024-10-01,31,"2,939227238","4.036.357.184,84",1030
163,13.077.415/0001-05,BB PREVIDENCIÁRIO FLUXO RENDA FIXA SIMPLES FUN...,2024-11-01,29,"2,960462845","4.217.186.353,15",1045
164,13.077.415/0001-05,BB PREVIDENCIÁRIO FLUXO RENDA FIXA SIMPLES FUN...,2024-12-01,31,"2,984656712","4.174.516.271,82",1035
165,13.077.415/0001-05,BB PREVIDENCIÁRIO FLUXO RENDA FIXA SIMPLES FUN...,2025-01-01,31,"3,012876808","4.468.412.591,20",1058


In [35]:
testing['Mês'] = pd.to_datetime(testing['Mês'], format='%m/%Y')
#testing['Mês'] = testing['Mês'].dt.strftime('%m/%Y')

In [38]:
testing_sorted = testing.sort_values(by='Mês', ignore_index=True)
testing_sorted

,Cnpj,Nome,Mês,Dia,Quota,Patrimônio Líquido,Número de Cotistas
0,13.077.415/0001-05,BB PREVIDENCIÁRIO FLUXO RENDA FIXA SIMPLES FUN...,2011-04-01,29,"1,000378","1.156.043,65",6
1,13.077.415/0001-05,BB PREVIDENCIÁRIO FLUXO RENDA FIXA SIMPLES FUN...,2011-05-01,31,"1,009178","43.118.555,31",41
2,13.077.415/0001-05,BB PREVIDENCIÁRIO FLUXO RENDA FIXA SIMPLES FUN...,2011-06-01,30,"1,018104","51.178.462,87",68
3,13.077.415/0001-05,BB PREVIDENCIÁRIO FLUXO RENDA FIXA SIMPLES FUN...,2011-07-01,29,"1,027238","86.750.252,95",93
4,13.077.415/0001-05,BB PREVIDENCIÁRIO FLUXO RENDA FIXA SIMPLES FUN...,2011-08-01,31,"1,037549","127.852.265,60",111
...,...,...,...,...,...,...,...
162,13.077.415/0001-05,BB PREVIDENCIÁRIO FLUXO RENDA FIXA SIMPLES FUN...,2024-10-01,31,"2,939227238","4.036.357.184,84",1030
163,13.077.415/0001-05,BB PREVIDENCIÁRIO FLUXO RENDA FIXA SIMPLES FUN...,2024-11-01,29,"2,960462845","4.217.186.353,15",1045
164,13.077.415/0001-05,BB PREVIDENCIÁRIO FLUXO RENDA FIXA SIMPLES FUN...,2024-12-01,31,"2,984656712","4.174.516.271,82",1035
165,13.077.415/0001-05,BB PREVIDENCIÁRIO FLUXO RENDA FIXA SIMPLES FUN...,2025-01-01,31,"3,012876808","4.468.412.591,20",1058
